<a href="https://colab.research.google.com/github/dvschultz/ai/blob/master/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

!git clone https://github.com/dvschultz/stylegan2
!pip install opensimplex # needed for noise interpolation
%cd stylegan2
%mkdir datasets

Cloning into 'stylegan2'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 215 (delta 0), reused 1 (delta 0), pack-reused 211
Receiving objects: 100% (215/215), 15.25 MiB | 31.86 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/stylegan2


#Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). I recommend doing this on your server because the files become quite large and will be slow to upload over FTP.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [0]:
python dataset_tool.py create_from_images ./datasets/my-custom-dataset ./my-custom-images

#Training
I do not recommend attempting to train your model on Colab. It will be a very tedious process. Here are the steps, however.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.


In [0]:
!python run_training.py --num-gpus=1 --data-dir=~/datasets --config=config-f --dataset=ffhq --mirror-augment=true

Once running, your training files will show up in the results folder.

#Test the model
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [0]:
!python run_generator.py generate-images --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Local submit - run_dir: results/00000-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/ladiesfloralcrop-network-snapshot-010237.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.
Generating image for seed 1 (0/25) ...
Generating image for seed 2 (1/25) ...
Generating image for seed 3 (2/25) ...
Generating image for seed 4 (3/25) ...
Generating image for seed 5 (4/25) ...
Generating image for seed 6 (5/25) ...
Generating image for seed 7 (6/25) ...
Generating image for seed 8 (7/25) ...
Generating image for seed 9 (8/25) ...
Generating image for seed 10 (9/25) ...
Generating image for seed 11 (10/25) ...
Generating image for seed 12 (11/25) ...
Generating image for seed 13 (12/25) ...
Generating image for seed 14 (13/25) ...
Generating image for seed 15 (14/25) ...
Generating imag

Let’s zip the generated files and download them.

In [0]:
!zip -r generated-0.7.zip /content/stylegan2/results/00000-generate-images

  adding: content/stylegan2/results/00000-generate-images/ (stored 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0025.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0014.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0007.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0018.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0017.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0010.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0015.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0024.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0021.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/seed0004.png (deflated 0%)
  adding: content/stylegan2/results/00000-generate-images/log.txt (deflated 74%

#Interpolation


In [0]:
!python run_generator.py generate-latent-walk --network=/content/ladiesfloralcrop-network-snapshot-010237.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

Local submit - run_dir: results/00002-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "/content/ladiesfloralcrop-network-snapshot-010237.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for step 0/204 ...
Generating image for step 1/204 ...
Generating image for step 2/204 ...
Generating image for step 3/204 ...
Generating image for step 4/204 ...
Generating image for step 5/204 ...
Generating image for step 6/204 ...
Generating image for step 7/204 ...
Generating image for step 8/204 ...
Generating image for step 9/204 ...
Generating image for step 10/204 ...
Generating image for step 11/204 ...
Generating image for step 12/204 ...
Generating image for step 13/204 ...
Generating image for step 14/204 ...
Generating image for step 15/204 ...
Generating image for step 16/204 ...
Generati

In [0]:
#convert to video 
!ffmpeg -r 24 -i ./results/00001-generate-latent-walk/step%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena